## 1. 데이터 로드

In [6]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc6_2_32 = pd.read_csv("../data/binutils_gcc3~9_op0~4_csv/"+'gcc6'+"_2_32.csv", index_col=0)

# 형태 출력
print(gcc6_2_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_2_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
gcc6_2_32.head()

(1905956, 2)
reset_index 완료
input data shape


,hex,bin
0,55,1
1,57,0
2,56,0
3,53,0
4,e8,0


## 2. Data 확인

In [7]:
# DATA 모양 확인
gcc6_2_32.values
print(gcc6_2_32.shape)

# 함수의 갯수
gcc6_2_32['bin'].value_counts()

# 함수 평균길이
gcc6_2_32['bin'].value_counts()[0]/gcc6_2_32['bin'].value_counts()[1]

array([['55', 1],
       ['57', 0],
       ['56', 0],
       ...,
       ['1c', 0],
       ['24', 0],
       ['c3', 0]], dtype=object)

(1905956, 2)


0    1894941
1      11015
Name: bin, dtype: int64

172.0327734906945

## 3. 함수의 갯수, 바이트 갯수의 빈공간 np.array 생성 

In [12]:
x = np.full((gcc6_2_32['bin'].value_counts()[1],70000), None, dtype='object')
x.shape
x.dtype

(11015, 70000)

dtype('O')

## 4. 함수 빈공간에 함수별로 한줄씩 값 대입

In [13]:
data = gcc6_2_32.values

row = -1
col = 0
col_list=[]

# data => [55,1], [89,1], [e5,1] ....
for i in data:
    # i[0] = 55, 89, e5... /  i[1] = 1, 0, 0, ....
    if i[1]==1:
        col_list.append(col) # col_list의 각 함수 길이들 넣는다.
        
        row+=1
        col=0
        x[row][col] = i[0]
#        print(row,col, end='/') # 오류났을 때 풀어보면 몇번째 행에서 값을 초과하는지 알 수 있음.
    else:
        x[row][col+1] = i[0]
        #if row >=1385:
        #    print(row,col, end=' ')
        col+=1
    
print('대입 완료')    

대입 완료


In [21]:
max(col_list)

67883

## 5. 최적의 x 찾기

In [15]:
# 예시
print('예시')
x[:, :6]
#print(len(col_list),'\n')

# 최소 3gram ~ 7gram 까지확인
for j in range(2, 7):
    tuple_ls = []
    for k in x[:, :j]:
        tuple_ls.append(tuple(k))

    set_ls = set(tuple_ls)
    print(j, 'gram일 때 ', len(set_ls),'쌍')

예시


array([['55', '57', '56', '53', 'e8', 'fc'],
       ['8b', '1c', '24', 'c3', None, None],
       ['55', '57', '89', 'd5', '56', '53'],
       ...,
       ['8b', '1c', '24', 'c3', None, None],
       ['55', '57', '56', '53', 'e8', 'fc'],
       ['8b', '1c', '24', 'c3', None, None]], dtype=object)

2 gram일 때  376 쌍
3 gram일 때  798 쌍
4 gram일 때  1173 쌍
5 gram일 때  1526 쌍
6 gram일 때  1789 쌍


In [20]:
# 임의 테스트
tls = []
for t in x[:, :4]:
    tls.append(tuple(t))
    
stls = set(stls)
stls

{('0f', 'b6', '73', '07'),
 ('83', 'c4', '04', '5b'),
 ('ff', '76', '18', 'ff'),
 ('c7', '83', '78', '00'),
 ('57', '56', '53', 'e8'),
 ('81', 'bd', '00', '01'),
 ('80', '7c', '24', '0b'),
 ('31', 'c9', '3b', '6c'),
 ('0f', 'b6', '47', '01'),
 ('56', '53', 'e8', 'fc'),
 ('55', '57', '56', '89'),
 ('c7', '85', '28', 'ff'),
 ('85', 'c0', '89', 'c2'),
 ('0f', 'b6', '41', '05'),
 ('3c', '5b', 'bf', '5d'),
 ('89', 'c2', '31', 'c0'),
 ('c7', '83', 'fc', '00'),
 ('b8', '55', '00', '00'),
 ('57', '56', '89', 'd7'),
 ('c7', '83', '34', '00'),
 ('39', 'ef', '89', '4c'),
 ('31', 'd2', 'b8', '31'),
 ('80', '7c', '24', '5c'),
 ('53', '31', 'db', '83'),
 ('83', 'e2', '08', '85'),
 ('c7', '83', '24', '00'),
 ('b8', '02', '00', '00'),
 ('8b', '50', '0c', '80'),
 ('83', 'ec', '0c', '50'),
 ('c7', '83', '50', '00'),
 ('8b', '44', '24', '34'),
 ('0f', 'b6', '41', '01'),
 ('c7', '46', '40', '03'),
 ('8b', '5c', '24', '04'),
 ('83', 'fe', '01', '0f'),
 ('8b', '83', '54', '00'),
 ('83', '7c', '24', '08'),
 